In [0]:
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN

In [0]:
PATH_TO_ANNOT = "/content/drive/My Drive/LicensePlateProject/license_plates_detection_train/annots"

In [0]:
class licensePateDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define one class
		self.add_class("dataset", 1, "licencePlate")
		# define data locations
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annots/'
		# find all images
		for filename in listdir(images_dir):
			# extract image id
			image_id = filename[:-4]
			# skip all images after 800 if we are building the train set
			if is_train and int(image_id) >= 720:
				continue
			# skip all images before 800 if we are building the validation set
			if not is_train and int(image_id) < 720:
				continue
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)
 
	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
         tree = ElementTree.parse(filename)
        # get the root of the document
         root = tree.getroot()
        # extract each bounding box
         boxes = list()
         for box in root.findall('.//bndbox'):
             xmin = int(box.find('xmin').text)
             ymin = int(box.find('ymin').text)
             xmax = int(box.find('xmax').text)
             ymax = int(box.find('ymax').text)
             coors = [xmin, ymin, xmax, ymax]
             boxes.append(coors)
        # extract image dimensions
         width = int(root.find('.//size/width').text)
         height = int(root.find('.//size/height').text)
         return boxes, width, height
 
	# load the masks for an image
	def load_mask(self, image_id):
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			masks[row_s:row_e, col_s:col_e, i] = 1
			class_ids.append(self.class_names.index('licencePlate'))
		return masks, asarray(class_ids, dtype='int32')
 
	# load an image reference
	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

In [0]:
# define a configuration for the model
class LicensePlateConfig(Config):
	# define the name of the configuration
	NAME = "LicensePlate_cfg"
	# number of classes (background + License Plate)
	NUM_CLASSES = 1 + 1
	# number of training steps per epoch
	STEPS_PER_EPOCH = 131
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

In [0]:
# train set
train_set = licensePateDataset()
train_set.load_dataset('/content/drive/My Drive/LicensePlateProject/license_plates_detection_train', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
 
# test/val set
test_set = licensePateDataset()
test_set.load_dataset('/content/drive/My Drive/LicensePlateProject/license_plates_detection_train', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

Train: 719
Test: 181


In [0]:
# prepare config
config = LicensePlateConfig()

In [0]:
# define the model
model = MaskRCNN(mode='training', model_dir='/content/drive/My Drive/LicensePlateProject', config=config)

In [0]:
model.load_weights('/content/drive/My Drive/LicensePlateProject/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

In [0]:
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=32, layers='heads')